Leyendo el archivo

In [16]:

import pandas as pd
from datetime import datetime
import os

folderCSV = './csv'
archivos = os.listdir(folderCSV)
print(archivos)
print(len(archivos))
archivo = archivos[(len(archivos)) - 1]
print(archivo)

df = pd.read_csv(f'csv\{archivo}')



['DescargaMasiva_1747149382298.csv', 'DescargaMasiva_1747322247770.csv', 'DescargaMasiva_1747419620988.csv', 'DescargaMasiva_1747670573511.csv', 'DescargaMasiva_1747756506776.csv', 'DescargaMasiva_1747848774020.csv', 'DescargaMasiva_1747923340552.csv', 'DescargaMasiva_1747927370332.csv', 'DescargaMasiva_1748009635513.csv', 'DescargaMasiva_1748012833135.csv', 'DescargaMasiva_1748271879681.csv', 'DescargaMasiva_1748359299230.csv', 'DescargaMasiva_1748455147845.csv', 'DescargaMasiva_1748534291986.csv', 'DescargaMasiva_1748624518461.csv', 'DescargaMasiva_1748879608611.csv']
16
DescargaMasiva_1748879608611.csv


Creando la limpieza de datos

In [17]:
#Dando formato de fecha a la columna Tiempo
df['Tiempo'] = pd.to_datetime(df['Tiempo'], format='%Y%m%d')

#Limpiando espacios en nombres de columnas
df.columns = df.columns.str.strip()

# ---------- Agregando los ID Tienda faltantes de Credraui ----------

# Aplicando el filtro
mascara = (df['Cadena'] == 'Chedraui') & (
  (df['ID TIENDA'].isna()) |  # Detecta NaN/None
  (df['ID TIENDA'].str.strip() == "") |  # Detecta cadenas vacías
  (df['ID TIENDA'] == 'SIN ASIGNAR')  # Detecta "SIN ASIGNAR"
)

# Aplicando el llenado
df.loc[mascara, 'ID TIENDA'] = 'CHE' + df.loc[mascara, 'NumTienda'].astype(str)

# ---------- Agregando los ID Tienda faltantes de City_Fresko ----------

# Aplicando el filtro
mascara = (df['Cadena'] == 'City_Fresko') & (
  (df['ID TIENDA'].isna()) |  # Detecta NaN/None
  (df['ID TIENDA'].str.strip() == "") |  # Detecta cadenas vacías
  (df['ID TIENDA'] == 'SIN ASIGNAR')  # Detecta "SIN ASIGNAR"
)

# Aplicando el llenado
df.loc[mascara, 'ID TIENDA'] = 'FRE' + df.loc[mascara, 'NumTienda'].astype(str)

# ---------- Agregando los ID Tienda faltantes de HEB ----------

# Aplicando el filtro
mascara = (df['Cadena'] == 'HEB') & (
  (df['ID TIENDA'].isna()) |  # Detecta NaN/None
  (df['ID TIENDA'].str.strip() == "") |  # Detecta cadenas vacías
  (df['ID TIENDA'] == 'SIN ASIGNAR')  # Detecta "SIN ASIGNAR"
)

# Aplicando el llenado
df.loc[mascara, 'ID TIENDA'] = 'HEB' + df.loc[mascara, 'NumTienda'].astype(str)

# ---------- Agregando los ID Tienda faltantes de Wal-Mart ----------

# Aplicando el filtro
mascara = (df['Cadena'] == 'Wal-Mart') & (
  (df['ID TIENDA'].isna()) |  # Detecta NaN/None
  (df['ID TIENDA'].str.strip() == "") |  # Detecta cadenas vacías
  (df['ID TIENDA'] == 'SIN ASIGNAR')  # Detecta "SIN ASIGNAR"
)

# Aplicando el llenado
df.loc[mascara, 'ID TIENDA'] = 'WM' + df.loc[mascara, 'NumTienda'].astype(str)



# ---------- Agregando y sustituyendo los nombres faltantes de Chedarui ----------
mascara = (df['NumTienda'] == 890)
df.loc[mascara, 'NomTienda'] = '30003 SUPERCITO SELECTO TLAX INSURGENTES CUAU 06-24'

mascara = (df['NumTienda'] == 839)
df.loc[mascara, 'NomTienda'] = '06701 CHEDRAUI SELECTO PUEBLA B CASCATTA'

mascara = (df['NumTienda'] == 276)
df.loc[mascara, 'NomTienda'] = '19210 SUPER CHEDRAUI PROGRESO YUC 11-24'


# ---------- Agregando y sustituyendo los nombres faltantes de City ----------
mascara = (df['NumTienda'] == 449)
df.loc[mascara, 'NomTienda'] = 'CITY MARKET LILAS'

mascara = (df['NumTienda'] == 451)
df.loc[mascara, 'NomTienda'] = 'FRESKO VALLE REAL'

mascara = (df['NumTienda'] == 453)
df.loc[mascara, 'NomTienda'] = 'LA COMER ALAMOS'

mascara = (df['NumTienda'] == 454)
df.loc[mascara, 'NomTienda'] = 'CITY MARKET LOPEZ MATEOS'

mascara = (df['NumTienda'] == 455)
df.loc[mascara, 'NomTienda'] = 'CITY MARKET SATELITE'

# df.loc[df['NumTienda'] == 455]

# df.loc[df['NumTienda'] == ]
# df.loc[df['ID TIENDA'].isna()]
# df.loc[df['ID TIENDA'] == 'SIN ASIGNAR']





In [18]:
# Guardando el archivo
fecha_actual = datetime.now().strftime('%d_%m_%Y')
nombre_archivo = f'InventarioLimpio_{fecha_actual}.csv'
df.to_csv(f'./limpio/{nombre_archivo}', index=False)

In [19]:
# Convertir a JSON y guardar en un archivo
# Formato ISO estándar
df['Tiempo'] = df['Tiempo'].dt.strftime('%d-%m-%Y')
df.to_json("./json/data.json", orient="records", indent=2)

In [20]:
import firebase_admin
from firebase_admin import credentials, db
import json

try:
    firebase_admin.get_app()
except ValueError:
    cred = credentials.Certificate("./firebase/serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://datainventario-37085-default-rtdb.firebaseio.com'
    })

# Carga tu JSON
with open("./json/data.json", "r") as f:
    data = json.load(f)

# Sube el JSON a la ruta 'datos' en la base de datos
ref = db.reference('/datos')
ref.set(data)

print("✅ JSON subido a Firebase Realtime Database")

✅ JSON subido a Firebase Realtime Database
